In [ ]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import numpy as np
import pandas as pd

import sys
sys.path.append("../../")
import src.soporte_logistica_preprocesamiento as pre

pd.set_option('display.float_format', '{:.2f}'.format)


# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestRegressor

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns',100)


In [2]:
df = pd.read_pickle("../../datos/01_datos_merge.plk")

# Gestión de Datos
- Vamos a eliminar columnas
- Convertir a categóricas si no tienen nulos
- Gestionar Nulos
- Convertir a categóricas tras gestionar nulso

# Eliminar Columnas

- EmployeeID: No aporta informacón
- EmployeeCount: Podemos eliminarla (solo un valor)
- StandardHours: Podemos Eliminarla (solo un valor)
- Over18 eliminamos al tener un solo valor

- PerformanceRating: La quitamos porque es una distribución de datos bastante mala, en una escala del 0 al 5 solo tenemos datos de 3 y 4, y hay pocos de 4

In [3]:
chao = ["EmployeeID"]
df.drop(chao, axis=1, inplace=True)

# Eliminar duplicados

In [4]:
df.shape

(4410, 28)

In [5]:
df.duplicated().sum()

np.int64(2837)

In [6]:
df.drop_duplicates(inplace=True)

In [7]:
df.duplicated().sum()

np.int64(0)

In [8]:
df.shape

(1573, 28)

In [9]:
cols_chao = ["EmployeeCount","StandardHours","Over18","PerformanceRating"]
df.drop(cols_chao, axis=1, inplace=True)

# Volver Categoría
### Sin agrupar:

- Education: Es categoría de 5 opciones
- JobLevel: Categoría de 5 opciones
   - Executive or senior management
   - Middle management
   - First-level management
   - Intermediate or experienced (senior staff)
   - Entry-level
- StockOptionLevel: Categoría de 4 opciones
- TrainingTimesLastYear: Categoría de 7 opciones

- JobInvolvement: Categoría de 4 opciones

- PercentSalaryHike: Categoría de 15 opciones
- DistanceFromHome: Es una categoría de 29 opciones

### Education
   - 1 'Below College'
   - 2 'College'
   - 3 'Bachelor'
   - 4 'Master'
   - 5 'Doctor'

In [10]:
df["Education"].unique()

array([2, 1, 4, 5, 3])

In [11]:
dictio_education = {
    1 : "Below College",
    2 : "College",
    3 : "Bachelor",
    4 : "Master",
    5 : "Doctor"
}
df["Education"] = df["Education"].map(dictio_education).astype("category")
df["Education"].unique()

['College', 'Below College', 'Master', 'Doctor', 'Bachelor']
Categories (5, object): ['Bachelor', 'Below College', 'College', 'Doctor', 'Master']

### JobLevel
   - Executive or senior management
   - Middle management
   - First-level management
   - Intermediate or experienced (senior staff)
   - Entry-level
   
Para saber que número pertenece a cuál, miraremos la media de sueldo

In [12]:
df["JobLevel"].unique()

array([1, 4, 3, 2, 5])

In [13]:
salario_por_nivel = df.groupby("JobLevel")["MonthlyIncome"].mean().reset_index()
salario_por_nivel.sort_values(by="MonthlyIncome",ascending=False)

,JobLevel,MonthlyIncome
3,4,77634.50
4,5,65908.27
1,2,65726.22
2,3,63639.21
0,1,62264.21


Como no tiene sentido, vamos a hacerlo más simple

In [14]:
dictio_job_level = {
    1 : "Level 1",
    2 : "Level 2",
    3 : "Level 3",
    4 : "Level 4",
    5 : "Level 5"
}
df["JobLevel"] = df["JobLevel"].map(dictio_job_level).astype("category")
df["JobLevel"].unique()

['Level 1', 'Level 4', 'Level 3', 'Level 2', 'Level 5']
Categories (5, object): ['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5']

### StockOptionLevel
- Miremos el Sueldo medio

In [15]:
df["StockOptionLevel"].unique()

array([0, 1, 3, 2])

In [16]:
salario_por_nivel = df.groupby("StockOptionLevel")["MonthlyIncome"].mean().reset_index()
salario_por_nivel.sort_values(by="MonthlyIncome",ascending=False)

,StockOptionLevel,MonthlyIncome
2,2,65980.06
1,1,65750.72
0,0,64168.74
3,3,63730.22


Como no tiene sentido, vamos a hacerlo más simple

In [17]:
dictio_stock_level = {
    0 : "Level 0",
    1 : "Level 1",
    2 : "Level 2",
    3 : "Level 3"
}
df["StockOptionLevel"] = df["StockOptionLevel"].map(dictio_stock_level).astype("category")
df["StockOptionLevel"].unique()

['Level 0', 'Level 1', 'Level 3', 'Level 2']
Categories (4, object): ['Level 0', 'Level 1', 'Level 2', 'Level 3']

### TrainingTimesLastYear

In [18]:
df["TrainingTimesLastYear"].unique()

array([6, 3, 2, 5, 4, 0, 1])

In [19]:
df["TrainingTimesLastYear"] = df["TrainingTimesLastYear"].astype("int").apply(lambda x: str(x)+" Times").astype("category")

In [20]:
df["TrainingTimesLastYear"].unique()

['6 Times', '3 Times', '2 Times', '5 Times', '4 Times', '0 Times', '1 Times']
Categories (7, object): ['0 Times', '1 Times', '2 Times', '3 Times', '4 Times', '5 Times', '6 Times']

### JobInvolvement
- 1 'Low' 
- 2 'Medium' 
- 3 'High' 
- 4 'Very High'


In [21]:
df["JobInvolvement"].unique()

array([3, 2, 1, 4])

In [22]:
dictio_job_inv = {
    1 : "Low",
    2 : "Medium",
    3 : "High",
    4 : "Very High"
}
df["JobInvolvement"] = df["JobInvolvement"].map(dictio_job_inv).astype("category")
df["JobInvolvement"].unique()

['High', 'Medium', 'Low', 'Very High']
Categories (4, object): ['High', 'Low', 'Medium', 'Very High']

### PercentSalaryHike


In [23]:
df["PercentSalaryHike"] = df["PercentSalaryHike"].apply(lambda x: str(x)+" %").astype("category")

In [24]:
df["PercentSalaryHike"].unique()

['11 %', '23 %', '15 %', '12 %', '13 %', ..., '16 %', '18 %', '19 %', '24 %', '25 %']
Length: 15
Categories (15, object): ['11 %', '12 %', '13 %', '14 %', ..., '22 %', '23 %', '24 %', '25 %']

### DistanceFromHome

In [25]:
df["DistanceFromHome"] = df["DistanceFromHome"].apply(lambda x: str(x)+" km").astype("category")
df["DistanceFromHome"].unique()

['6 km', '10 km', '17 km', '2 km', '8 km', ..., '21 km', '26 km', '27 km', '12 km', '23 km']
Length: 29
Categories (29, object): ['1 km', '10 km', '11 km', '12 km', ..., '6 km', '7 km', '8 km', '9 km']

# Gestionar Nulos y Volver Categoría
- NumCompaniesWorked: Categórica de 11 opciones (19 Nulos), con nulos no hacemos nada aún.
- EnvironmentSatisfaction: Categoría de 5 opciones (25 nulos)
- JobSatisfaction: Categoría de 5 opciones (20 nulos)
- WorkLifeBalance: Categoría de 5 opciones (38 nulos)

### Veamos las filas con nulos y su attrition:

In [26]:
cond1 = df["NumCompaniesWorked"].isnull() == True
cond2 = df["EnvironmentSatisfaction"].isnull() == True
cond3 = df["JobSatisfaction"].isnull() == True
cond4 = df["WorkLifeBalance"].isnull() == True
df[cond1 | cond2 | cond3 | cond4]["Attrition"].value_counts()

Attrition
No     82
Yes    13
Name: count, dtype: int64

### Que sabemos
- 102 Filas con nulos
- 88 No se han ido
- 14 Si se han ido
### Que hacemos
- Tenemos un desbalanceo muy grande, hay muchos No y pocos Si
- Eliminamos NO
- Gestionamos los Sí

In [27]:
cond1 = df["NumCompaniesWorked"].isnull() == True
cond2 = df["EnvironmentSatisfaction"].isnull() == True
cond3 = df["JobSatisfaction"].isnull() == True
cond4 = df["WorkLifeBalance"].isnull() == True
cond5 = df["Attrition"] == "No"
df.drop(df.loc[(cond1 | cond2 | cond3 | cond4 )& cond5].index,inplace=True)

### Gestión de los Sí

In [28]:
cond1 = df["NumCompaniesWorked"].isnull() == True
cond2 = df["EnvironmentSatisfaction"].isnull() == True
cond3 = df["JobSatisfaction"].isnull() == True
cond4 = df["WorkLifeBalance"].isnull() == True
df[cond1 | cond2 | cond3 | cond4]["Attrition"].value_counts()

Attrition
Yes    13
Name: count, dtype: int64

### Rellenar Nulos con Random Forest

In [29]:
df.reset_index(drop=True,inplace=True)

In [30]:
df.isnull().sum()

Age                        0
Attrition                  0
BusinessTravel             0
Department                 0
DistanceFromHome           0
Education                  0
EducationField             0
Gender                     0
JobLevel                   0
JobRole                    0
MaritalStatus              0
MonthlyIncome              0
NumCompaniesWorked         4
PercentSalaryHike          0
StockOptionLevel           0
TotalWorkingYears          8
TrainingTimesLastYear      0
YearsAtCompany             0
YearsSinceLastPromotion    0
YearsWithCurrManager       0
EnvironmentSatisfaction    4
JobSatisfaction            1
WorkLifeBalance            4
JobInvolvement             0
dtype: int64

In [31]:
df_num = df.select_dtypes(include=np.number)

# Random Forest
iterative_imputer = IterativeImputer(
    estimator=RandomForestRegressor(random_state=42),
    random_state=42
)

df_num_imputado = iterative_imputer.fit_transform(df_num)
df_num_sin_nulos = pd.DataFrame(df_num_imputado, columns=df_num.columns)
df[df_num.columns] = df_num_sin_nulos
df.isnull().sum()


Age                        0
Attrition                  0
BusinessTravel             0
Department                 0
DistanceFromHome           0
Education                  0
EducationField             0
Gender                     0
JobLevel                   0
JobRole                    0
MaritalStatus              0
MonthlyIncome              0
NumCompaniesWorked         0
PercentSalaryHike          0
StockOptionLevel           0
TotalWorkingYears          0
TrainingTimesLastYear      0
YearsAtCompany             0
YearsSinceLastPromotion    0
YearsWithCurrManager       0
EnvironmentSatisfaction    0
JobSatisfaction            0
WorkLifeBalance            0
JobInvolvement             0
dtype: int64

### Categorizar
- NumCompaniesWorked: Categórica de 11 opciones (19 Nulos), con nulos no hacemos nada aún.
- EnvironmentSatisfaction: Categoría de 5 opciones (25 nulos)
- JobSatisfaction: Categoría de 5 opciones (20 nulos)
- WorkLifeBalance: Categoría de 5 opciones (38 nulos)

In [32]:
df["NumCompaniesWorked"].value_counts()

NumCompaniesWorked
1.00    527
0.00    200
3.00    159
2.00    147
4.00    141
7.00     77
6.00     72
5.00     64
9.00     52
8.00     49
4.60      1
4.16      1
4.75      1
Name: count, dtype: int64

### Corregir
Debemos poner los que puso Random Forest en decimal, en enteros

In [33]:
df.loc[df["NumCompaniesWorked"].between(4.5,5.49,"both"),"NumCompaniesWorked"] = 5
df.loc[df["NumCompaniesWorked"].between(3.5,4.49,"both"),"NumCompaniesWorked"] = 4
df.loc[df["NumCompaniesWorked"].between(0.5,1.49,"both"),"NumCompaniesWorked"] = 1

### Convertir en categoría

In [34]:
df["NumCompaniesWorked"] = df["NumCompaniesWorked"].astype("int")
df["NumCompaniesWorked"] = df["NumCompaniesWorked"].apply(lambda x: str(x)+" companies")
df.loc[df["NumCompaniesWorked"] == "0 companies","NumCompaniesWorked"] = "None"
df["NumCompaniesWorked"] = df["NumCompaniesWorked"].astype("category")
df["NumCompaniesWorked"].unique()

['1 companies', 'None', '3 companies', '4 companies', '2 companies', '7 companies', '9 companies', '5 companies', '6 companies', '8 companies']
Categories (10, object): ['1 companies', '2 companies', '3 companies', '4 companies', ..., '7 companies', '8 companies', '9 companies', 'None']

### EnvironmentSatisfaction


In [35]:
df["EnvironmentSatisfaction"].value_counts()

EnvironmentSatisfaction
3.00    460
4.00    450
2.00    291
1.00    286
2.23      1
3.71      1
1.55      1
1.46      1
Name: count, dtype: int64

In [36]:
df.loc[df["EnvironmentSatisfaction"].between(0.5,1.49,"both"),"EnvironmentSatisfaction"] = 1
df.loc[df["EnvironmentSatisfaction"].between(1.5,2.49,"both"),"EnvironmentSatisfaction"] = 2
df.loc[df["EnvironmentSatisfaction"].between(3.5,4.49,"both"),"EnvironmentSatisfaction"] = 4
df["EnvironmentSatisfaction"].value_counts()

EnvironmentSatisfaction
3.00    460
4.00    451
2.00    293
1.00    287
Name: count, dtype: int64

In [37]:
dictio_satisf = {
    1 :'Low',
    2 : 'Medium', 
    3 : 'High', 
    4 : 'Very High'
}

df["EnvironmentSatisfaction"] = df["EnvironmentSatisfaction"].map(dictio_satisf).astype("category")
df["EnvironmentSatisfaction"].unique()


['High', 'Medium', 'Very High', 'Low']
Categories (4, object): ['High', 'Low', 'Medium', 'Very High']

### JobSatisfaction


In [38]:
df["JobSatisfaction"].value_counts()

JobSatisfaction
4.00    464
3.00    450
1.00    292
2.00    284
1.57      1
Name: count, dtype: int64

In [39]:
df.loc[df["JobSatisfaction"].between(0.5,1.99,"both"),"JobSatisfaction"] = 1
df["JobSatisfaction"] = df["JobSatisfaction"].map(dictio_satisf).astype("category")
df["JobSatisfaction"].unique()

['Very High', 'Medium', 'Low', 'High']
Categories (4, object): ['High', 'Low', 'Medium', 'Very High']

### WorkLifeBalance

In [40]:
df["WorkLifeBalance"].value_counts()

WorkLifeBalance
3.00    904
2.00    347
4.00    154
1.00     82
2.89      1
2.17      1
2.25      1
2.16      1
Name: count, dtype: int64

In [41]:
df.loc[df["WorkLifeBalance"].between(1.5,2.49,"both"),"WorkLifeBalance"] = 2
df.loc[df["WorkLifeBalance"].between(2.5,3.49,"both"),"WorkLifeBalance"] = 3
df["WorkLifeBalance"] = df["WorkLifeBalance"].map(dictio_satisf).astype("category")
df["WorkLifeBalance"].unique()

['Medium', 'Very High', 'Low', 'High']
Categories (4, object): ['High', 'Low', 'Medium', 'Very High']

In [42]:
df

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement
0,51.00,No,Travel_Rarely,Sales,6 km,College,Life Sciences,Female,Level 1,Healthcare Representative,Married,131160.00,1 companies,11 %,Level 0,1.00,6 Times,1.00,0.00,0.00,High,Very High,Medium,High
1,31.00,Yes,Travel_Frequently,Research & Development,10 km,Below College,Life Sciences,Female,Level 1,Research Scientist,Single,41890.00,None,23 %,Level 1,6.00,3 Times,5.00,1.00,4.00,High,Medium,Very High,Medium
2,32.00,No,Travel_Frequently,Research & Development,17 km,Master,Other,Male,Level 4,Sales Executive,Married,193280.00,1 companies,15 %,Level 3,5.00,2 Times,5.00,0.00,3.00,Medium,Medium,Low,High
3,38.00,No,Non-Travel,Research & Development,2 km,Doctor,Life Sciences,Male,Level 3,Human Resources,Married,83210.00,3 companies,11 %,Level 3,13.00,5 Times,8.00,7.00,5.00,Very High,Very High,High,Medium
4,32.00,No,Travel_Rarely,Research & Development,10 km,Below College,Medical,Male,Level 1,Sales Executive,Single,23420.00,4 companies,12 %,Level 2,9.00,2 Times,6.00,0.00,4.00,Very High,Low,High,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1486,42.00,No,Travel_Rarely,Research & Development,23 km,Master,Other,Male,Level 1,Research Scientist,Married,23730.00,6 companies,14 %,Level 1,21.00,3 Times,1.00,0.00,0.00,Very High,High,Very High,Very High
1487,50.00,No,Travel_Frequently,Research & Development,25 km,Master,Life Sciences,Female,Level 2,Healthcare Representative,Married,32020.00,2 companies,24 %,Level 1,32.00,2 Times,2.00,2.00,2.00,Very High,Low,High,High
1488,45.00,No,Travel_Frequently,Research & Development,19 km,Bachelor,Medical,Male,Level 3,Laboratory Technician,Single,36810.00,3 companies,17 %,Level 1,22.00,2 Times,0.00,0.00,0.00,Medium,Medium,High,High
1489,28.00,Yes,Travel_Rarely,Human Resources,1 km,Bachelor,Medical,Female,Level 2,Laboratory Technician,Married,24130.00,1 companies,16 %,Level 1,1.00,2 Times,1.00,0.00,0.00,High,Medium,High,High


# Corregir tipos de información

In [43]:
df["TrainingTimesLastYear"].unique()

['6 Times', '3 Times', '2 Times', '5 Times', '4 Times', '0 Times', '1 Times']
Categories (7, object): ['0 Times', '1 Times', '2 Times', '3 Times', '4 Times', '5 Times', '6 Times']

In [44]:
cols_int = ["Age","TotalWorkingYears","TotalWorkingYears","YearsAtCompany","YearsSinceLastPromotion","YearsWithCurrManager","MonthlyIncome"]
for col in cols_int:
    print(col)
    df[col] = df[col].astype(int)

Age
TotalWorkingYears
TotalWorkingYears
YearsAtCompany
YearsSinceLastPromotion
YearsWithCurrManager
MonthlyIncome


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1491 entries, 0 to 1490
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   Age                      1491 non-null   int64   
 1   Attrition                1491 non-null   object  
 2   BusinessTravel           1491 non-null   object  
 3   Department               1491 non-null   object  
 4   DistanceFromHome         1491 non-null   category
 5   Education                1491 non-null   category
 6   EducationField           1491 non-null   object  
 7   Gender                   1491 non-null   object  
 8   JobLevel                 1491 non-null   category
 9   JobRole                  1491 non-null   object  
 10  MaritalStatus            1491 non-null   object  
 11  MonthlyIncome            1491 non-null   int64   
 12  NumCompaniesWorked       1491 non-null   category
 13  PercentSalaryHike        1491 non-null   category
 14  StockOpt

Pasamos los object a category

In [46]:
cols_cat = ["BusinessTravel","Department","EducationField","Gender","JobRole","MaritalStatus"]
for cat in cols_cat:
    df[cat] = df[cat].astype("category")
    

# Pasamos VR a numérica
- No: 0
- Sí: 1

Donde no, es que sigue en la empresa

Donde sí, es que se fue de la empresa

In [47]:
df.sample()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement
1329,41,No,Travel_Frequently,Sales,10 km,College,Medical,Female,Level 3,Manufacturing Director,Divorced,79450,4 companies,12 %,Level 1,21,6 Times,7,7,7,Very High,Low,Medium,Medium


In [48]:
dictio_vr = {
    "No" : 0,
    "Yes" : 1
}

df["Attrition"] = df["Attrition"].map(dictio_vr).astype("category")

In [49]:
df.sample()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement
64,30,0,Travel_Rarely,Research & Development,13 km,Bachelor,Life Sciences,Female,Level 4,Research Director,Married,41970,1 companies,13 %,Level 0,11,3 Times,10,1,9,Very High,Very High,High,High


# Guardamos el DF

In [50]:
df.to_pickle("../../datos/02_datos_gestionados.plk")